# Verse Interpreter development


Essential installs:

- `pip3 install .....`

This version is used to test the first steps for the verse interpreter. The final version will be used as a full python file.


# ERROR-TYPE ENUMERATION


In [240]:
from enum import Enum

class ErrorType(Enum):
    SyntaxError = 'Wrong Syntax at'
    SemanticError = 'Wrong Semantics at'
    UnkownError = 'Operation Failure'

# LOGGER CLASSS


In [241]:
class Logger:
    def __init__(self):{}
        
    def __log__(self, string:str):{}

    def __log_error__(self,string:str, type:ErrorType):{}

In [242]:
class Console_Logger(Logger):

    def __log__(self, string:str):
        print(string)

    def __log_error__(self,string:str, type:ErrorType):       
        print("ERROR| " + type.value + ": " + string)


In [243]:
import string



class TokenTypes(Enum):
    # Data
    INTEGER = int
    IDENTIFIER = string #Names/Variables
    INT_TYPE = "int"
    TUPLE_TYPE = "tuple"
    ARRAY_TYPE = "array"
    FAIL = "false?"
    # Aritmetics
    PLUS = "+"
    MINUS = "-"
    MULTIPLY = "*"
    DIVIDE = "/"
    GREATER = ">"
    GREATEREQ = ">="
    LOWER = "<"
    LOWEREQ = "<="
    CHOICE = "|"
    LAMBDA = "=>"
    # Mehtods
    FOR = "for"
    DO = "do"
    IF = "if"
    THEN = "then"
    ELSE = "else"
    # Else
    EOF = ""
    COLON = ":"
    COMMA=","
    SEMICOLON =";"
    BINDING =":="
    LBRACKET = "("
    RBRACKET = ")"
    SBL = "["
    SBR = "]"
    CBL = "{"
    CBR = "}"
    EQUAL = "="
    SCOPE = ":"
    DOT = "."
    

# File Reader


In [244]:
class FileReader:
    def __init__(self):{}

    
    def get_Lines(self, name:str):
       
        try:
            f = open('..\modules\{}'.format(name),'r')
            lines = f.read().split("\n")
            return (lines,True)
        except:
            return ([],False)
        
       

   
reader = FileReader()
segments,read_success = reader.get_Lines('example.txt')
print(segments)

['halloo wie geht', 'ws', 'efaf', ' aefe ', ' eeerwre']


In [245]:
import string

class NodeStatus(Enum):
    # Data
    VALUE_RECEIVABLE = None
    NOT_ASSIGNED_YET = None
    ERROR = None
   
    

In [246]:
class VisitorNode:
    def __init__(self, type:NodeStatus, node):
        self.type = type
        self.node = node

In [247]:
class Token:
    def __init__(self, type: TokenTypes, value) -> None:
        self.type = type
        self.value = value
    
    def __info__(self):
         return "{}: {}".format(self.type, self.value)

In [248]:
class lexicon:
    def __init__(self, input: string):
        self.input = input
        self.index = 0
        self.current_char = self.input[self.index]
    
    # moves the pointer a character forward
    def forward(self) -> None:
        self.index += 1

        # checks if index is out of range
        if self.index >= len(self.input):
            self.current_char = None
            return
        
        self.current_char = self.input[self.index]
    
    def backward(self) -> None:
        self.index -= 1

        # checks if index is out of range
        if self.index < 0:
            self.current_char = None
            return
        
        self.current_char = self.input[self.index]
    
    def get_int(self) -> int:
        if self.index >= len(self.input):
            return None
        
        result = self.input[self.index]

        # checks if there are multiple digits
        while True:
            self.forward()

            if self.index < len(self.input) and self.input[self.index] != None and self.input[self.index].isnumeric():
                result += self.input[self.index]
            else:
                self.backward()
                break

        return int(result)
    
    def get_var(self) -> string:
        if self.index >= len(self.input):
            return None
        
        result = self.input[self.index]

        # checks if there is a longer variable name
        while True:
            self.forward()

            if self.index < len(self.input) and self.input[self.index] != None and self.input[self.index].isalpha():
                result += self.input[self.index]
            elif self.index < len(self.input) and self.input[self.index] != None and self.input[self.index] == '?':
                result += self.input[self.index]
            else:
                self.backward()
                break
        
        return result
    
    def get_a_string_from_input(self) -> string:
        result = self.input[self.index]
        self.forward()
        
        if self.index < len(self.input) and self.input[self.index] != None:
                result += self.input[self.index]
        return result
    

    def get_binding(self):
        if self.index >= len(self.input) and self.index + 1 >= len(self.input):
            return None
        
        result = self.get_a_string_from_input()

        match result:
            case TokenTypes.BINDING.value:
                return Token(TokenTypes.BINDING, TokenTypes.BINDING.value)
        
        self.backward()
        return Token(TokenTypes.COLON, TokenTypes.COLON.value)
    
    def get_greater_eq(self):
        if self.index >= len(self.input) and self.index + 1 >= len(self.input):
            return None
        
        result = self.get_a_string_from_input()

        match result:
            case TokenTypes.GREATEREQ.value:
                return Token(TokenTypes.GREATEREQ, TokenTypes.GREATEREQ.value)
        
        self.backward()
        return Token(TokenTypes.GREATER, TokenTypes.GREATER.value)
    

    def get_lower_eq(self):
        if self.index >= len(self.input) and self.index + 1 >= len(self.input):
            return None
        
        result = self.input[self.index]
        self.forward()
        
        result = self.get_a_string_from_input()

        match result:
            case TokenTypes.LOWEREQ.value:
                return Token(TokenTypes.LOWEREQ, TokenTypes.LOWEREQ.value)
        
        self.backward()
        return Token(TokenTypes.LOWER, TokenTypes.LOWER.value)
    
    def get_lambda(self):
        if self.index >= len(self.input) and self.index + 1 >= len(self.input):
            return None
        
        result = self.input[self.index]
        
        result = self.get_a_string_from_input()

        match result:
            case TokenTypes.LAMBDA.value:
                return Token(TokenTypes.LAMBDA, TokenTypes.LAMBDA.value)
        
        self.backward()
        return Token(TokenTypes.EQUAL, TokenTypes.EQUAL.value)
    
   
    def get_token(self, char: string) -> Token:
        token = self.check_for_tokentypes(char)

        if token.type != TokenTypes.EOF:
            return token
        
        if char == None:
            return token
            
        # skip spaces.
        if char == ' ':
            self.forward()
            return self.get_token(self.current_char)
        
        if char == TokenTypes.COLON:
            return self.get_next(TokenTypes.BINDING.value)

        # checks if the current character is a number.
        if char.isnumeric():
            result = self.get_int()
            return Token(TokenTypes.INTEGER, result)
        
        if char.isalpha():
            result = self.get_var()
            token = self.check_for_tokentypes(result)
            if(token.type == TokenTypes.EOF):
                token = Token(TokenTypes.IDENTIFIER, result)  
                  
        return token

    def check_for_tokentypes(self, char: string) -> Token:
        # checks if the current character is a supported token type.
        match char:
            case TokenTypes.INTEGER.value:
                return Token(TokenTypes.INTEGER, TokenTypes.INTEGER.value)
            case TokenTypes.IDENTIFIER.value:
                return Token(TokenTypes.IDENTIFIER, TokenTypes.IDENTIFIER.value)
            case TokenTypes.INT_TYPE.value:
                return Token(TokenTypes.INT_TYPE, TokenTypes.INT_TYPE.value)
            case TokenTypes.TUPLE_TYPE.value:
                return Token(TokenTypes.TUPLE_TYPE, TokenTypes.TUPLE_TYPE.value)
            case TokenTypes.ARRAY_TYPE.value:
                return Token(TokenTypes.ARRAY_TYPE, TokenTypes.ARRAY_TYPE.value)
            case TokenTypes.FAIL.value:
                return Token(TokenTypes.FAIL, TokenTypes.FAIL.value)
            case TokenTypes.PLUS.value:
                return Token(TokenTypes.PLUS, TokenTypes.PLUS.value)
            case TokenTypes.MINUS.value:
                return Token(TokenTypes.MINUS, TokenTypes.MINUS.value)
            case TokenTypes.MULTIPLY.value:
                return Token(TokenTypes.MULTIPLY, TokenTypes.MULTIPLY.value)
            case TokenTypes.DIVIDE.value:
                return Token(TokenTypes.DIVIDE, TokenTypes.DIVIDE.value)
            case TokenTypes.GREATER.value:
                return self.get_greater_eq()
            case TokenTypes.GREATEREQ.value:
                return Token(TokenTypes.GREATEREQ, TokenTypes.GREATEREQ.value)
            case TokenTypes.LOWER.value:
                return self.get_lower_eq()
            case TokenTypes.LOWEREQ.value:
                return Token(TokenTypes.LOWEREQ, TokenTypes.LOWEREQ.value)
            case TokenTypes.CHOICE.value:
                return Token(TokenTypes.CHOICE, TokenTypes.CHOICE.value)
            case TokenTypes.FOR.value:
                return Token(TokenTypes.FOR, TokenTypes.FOR.value)
            case TokenTypes.DO.value:
                return Token(TokenTypes.DO, TokenTypes.DO.value)
            case TokenTypes.IF.value:
                return Token(TokenTypes.IF, TokenTypes.IF.value)
            case TokenTypes.THEN.value:
                return Token(TokenTypes.THEN, TokenTypes.THEN.value)
            case TokenTypes.ELSE.value:
                return Token(TokenTypes.ELSE, TokenTypes.ELSE.value)
            case TokenTypes.EOF.value:
                return Token(TokenTypes.EOF, TokenTypes.EOF.value)
            case TokenTypes.COLON.value:
                return self.get_binding()
            case TokenTypes.COMMA.value:
                return Token(TokenTypes.COMMA, TokenTypes.COMMA.value)
            case TokenTypes.SEMICOLON.value:
                return Token(TokenTypes.SEMICOLON, TokenTypes.SEMICOLON.value)
            case TokenTypes.BINDING.value:
                return Token(TokenTypes.BINDING, TokenTypes.BINDING.value)
            case TokenTypes.LBRACKET.value:
                return Token(TokenTypes.LBRACKET, TokenTypes.LBRACKET.value)
            case TokenTypes.RBRACKET.value:
                return Token(TokenTypes.RBRACKET, TokenTypes.RBRACKET.value)
            case TokenTypes.RBRACKET.value:
                return Token(TokenTypes.RBRACKET, TokenTypes.RBRACKET.value)
            case TokenTypes.SBR.value:
                return Token(TokenTypes.SBR, TokenTypes.SBR.value)
            case TokenTypes.CBL.value:
                return Token(TokenTypes.CBL, TokenTypes.CBL.value)
            case TokenTypes.CBR.value:
                return Token(TokenTypes.CBR, TokenTypes.CBR.value)
            case TokenTypes.SBL.value:
                return Token(TokenTypes.SBL, TokenTypes.SBL.value)
            case TokenTypes.EQUAL.value:
                return self.get_lambda()
            case TokenTypes.SCOPE.value:
                return Token(TokenTypes.SCOPE, TokenTypes.SCOPE.value)
            case TokenTypes.DOT.value:
                return Token(TokenTypes.DOT, TokenTypes.DOT.value)
            case _:
                return Token(TokenTypes.EOF, None)

In [249]:
lexer = lexicon("if (x=0) then { y=3; z=4 } else { y=232; z=913 };")

while lexer.current_char is not None:
    token = lexer.get_token(lexer.current_char)
    print(str(token.value) + " is of the tokentype: " + str(token.type))
    lexer.forward()

if is of the tokentype: TokenTypes.IF
( is of the tokentype: TokenTypes.LBRACKET
x is of the tokentype: TokenTypes.IDENTIFIER
= is of the tokentype: TokenTypes.EQUAL
0 is of the tokentype: TokenTypes.INTEGER
) is of the tokentype: TokenTypes.RBRACKET
then is of the tokentype: TokenTypes.THEN
{ is of the tokentype: TokenTypes.CBL
y is of the tokentype: TokenTypes.IDENTIFIER
= is of the tokentype: TokenTypes.EQUAL
3 is of the tokentype: TokenTypes.INTEGER
; is of the tokentype: TokenTypes.SEMICOLON
z is of the tokentype: TokenTypes.IDENTIFIER
= is of the tokentype: TokenTypes.EQUAL
4 is of the tokentype: TokenTypes.INTEGER
} is of the tokentype: TokenTypes.CBR
else is of the tokentype: TokenTypes.ELSE
{ is of the tokentype: TokenTypes.CBL
y is of the tokentype: TokenTypes.IDENTIFIER
= is of the tokentype: TokenTypes.EQUAL
232 is of the tokentype: TokenTypes.INTEGER
; is of the tokentype: TokenTypes.SEMICOLON
z is of the tokentype: TokenTypes.IDENTIFIER
= is of the tokentype: TokenTypes.E

In [250]:


#Class that takes a parsed node, containes information if node could have been parsed
class ParsedNode:
    def __init__(self, node, hasSyntaxError:bool ):
        self.node = node
        self.hasSyntaxError = hasSyntaxError



In [251]:
import itertools


class BaseNode:
    def __init__(self, token) -> None:
        self.token = token
        self.current_scope = 0

    def visit(self, node):
        if isinstance(node, ProgramNode):
                return self.visit_programNode(node)
        elif isinstance(node, BlockNode):
                return self.visit_blockNode(node)
        elif isinstance(node, ScopeNode):
                return self.visit_scopeNode(node)
        elif isinstance(node, BindingNode):
                return self.visit_bindingNode(node)
        elif isinstance(node, OperatorNode):
                return self.visit_operatorNode(node)
        elif isinstance(node, NumberNode):
                return self.visit_numberNode(node)
        elif isinstance(node, FailNode):
                return self.visit_failNode(node)
        elif isinstance(node, UnaryNode):
                return self.visit_unaryNode(node)
        elif isinstance(node, ParsedNode):
             return self.visit(node.node)
        elif isinstance(node, ForNode):
            return self.visit_forNode(node)
        elif isinstance(node, ScopeNode):
            return self.visit_scopeNode(node)
        elif isinstance(node, ForNode):
            return self.visit_forNode(node)
        elif isinstance(node, SequenceNode):
            return self.visit_sequenceNode(node)
        
        

    def visit_blockNode(self, node):
            for n in node.nodes:
                return self.visit(n)
            
    def visit_programNode(self, node):
        return self.visit(node.node)

   
    def visit_scopeNode(self, node):
        # token:Token, nodes:list[BaseNode], type
        pass

    def visit_bindingNode(self, node):
        # token:Token, leftNode:IdentifierNode, rightNode:BaseNode
        identifierName:string = node.leftNode.token.type.value
        # Check if identifer exists outside its scope level

        # If not exists:
        val = self.visit(node.rightNode)
        if(val.type == NodeStatus.NOT_ASSIGNED_YET):
             return val
        # safe identifier wit val and its scope level
        pass

    def visit_numberNode(self, node):
        return VisitorNode(NodeStatus.VALUE_RECEIVABLE, NumberNode(node.token))
    
    def visit_operatorNode(self, node):

        val_left = self.visit(node.leftNode)
        if val_left.node.token.type == TokenTypes.FAIL:
                return val_left
        val_right = self.visit(node.rightNode)
        if val_right.node.token.type == TokenTypes.FAIL:
                return val_right
        
        return self.GetNodeForOperation(val_left,val_right,node.token)
        """
        match node.token.type:
            case TokenTypes.DIVIDE:
                return self.visit(node.leftNode) // self.visit(node.rightNode)
            case TokenTypes.MULTIPLY:
                return val_left.value * val_right.value
            case TokenTypes.PLUS:              
                return self.GetNodeForOperation(val_left,val_right)
            case TokenTypes.MINUS:
                return self.visit(node.leftNode) - self.visit(node.rightNode)       
            case TokenTypes.GREATER:
                if val_left.value > val_right.value:
                    return val_left
                else: return Value(ValueTypes.FAIL,ValueTypes.FAIL.value)
            case TokenTypes.LOWER:
                if val_left.value < val_right.value:
                    return val_left
                else: return Value(ValueTypes.FAIL,ValueTypes.FAIL.value)  
            case TokenTypes.DOT:
                return [i for i in range(self.visit(node.leftNode), self.visit(node.rightNode)+1)]
        """
    def GetNodeForOperation(self,val_left,val_right, opToken:Token):
        nodes = []
        nodeStatus = NodeStatus.VALUE_RECEIVABLE
        token = None

        if val_left.node.token.type == TokenTypes.CHOICE:
            token = val_left.node.token
            for n in val_left.node.nodes:
                if val_right.node.token.type == TokenTypes.CHOICE:
                    for n2 in val_right.node.nodes:
                       node = self.doOperation(n.value,n2.value,opToken)
                       nodes.append(node)
                else:nodes.append(self.doOperation(n.value,val_right.node.token.value,opToken)) 

        elif val_right.node.token.type == TokenTypes.CHOICE:
            token = val_right.node.token
            for n in val_right.node.nodes:
               nodes.append(self.doOperation(n.value,val_left.node.token.value,opToken)) 

        elif val_left.node.token.type == TokenTypes.INTEGER and val_right.node.token.type == TokenTypes.INTEGER:
             return VisitorNode(nodeStatus, NumberNode(Token(TokenTypes.INTEGER,val_left.node.token.value + val_right.node.token.value)))
        return VisitorNode(nodeStatus,SequenceNode(token, nodes))
                             
    def doOperation(self,val1:int,val2:int, token:Token):
        result = 0
        match token.type:
            case TokenTypes.DIVIDE:
                result = val1 // val2
            case TokenTypes.MULTIPLY:
                result = val1 * val2
            case TokenTypes.PLUS:              
                result = val1 + val2
            case TokenTypes.MINUS:
                result = val1 - val2      
            case TokenTypes.GREATER:
                if val1 > val2:
                    result = val1
                else: return FailNode(Token(TokenTypes.FAIL, TokenTypes.FAIL.value)) 
            case TokenTypes.LOWER:
                if val1:
                    result = val1
                else: return FailNode(Token(TokenTypes.FAIL, TokenTypes.FAIL.value)) 
        return  NumberNode(Token(TokenTypes.INTEGER, result))  
           
    def visit_unaryNode(self, node):
        mul:int = 1
        if node.token.type == TokenTypes.MINUS:
            mul = -1
        val = self.visit(node.node)
        if(val.type == ValueTypes.FAIL):
            return(val)
        return Value(ValueTypes.INTEGER, mul * self.visit(node.node).value)
    
    def visit_identifierNode(self, node):
        # Check in scope table if exists
        # if not return Value(ValueTypes.NOT_ASSIGNED_YET, node.token.type.value)
        # else return its value: Value(...)
        pass

    def visit_typeNode(self, node):
        
        pass

    def visit_sequenceTypeNode(self, node):
       pass
         

    def visit_argumentsNode(self, node):
        pass

    def visit_funcCallNode(self, node):
        pass

    def visit_ParamsNode(self, node):
        pass
    def visit_funcDeclNode(self, node):
        pass
    def visit_forNode(self, node):
        pass
    def visit_ifNode(self, node):
        pass
    def visit_rigidEqNode(self, node):
        # token:Token, left_node:BaseNode, right_node:BaseNode
        """
        val_left = self.visit(node.leftNode)
        if val_left.type == ValueTypes.FAIL:
                return val_left
        val_right = self.visit(node.rightNode)
        if val_right.type == ValueTypes.FAIL:
                return val_right
        if val_left.value == val_right:
            return val_left
        else: Value(ValueTypes.FAIL, ValueTypes.FAIL.value)
        """
        
    def visit_flexibleEqNode(self, node):
        # token:Token, left_node:BaseNode, right_node:BaseNode
        if(node.token.type == TokenTypes.IDENTIFIER):
             # Get type of node tuple(int,int) (21+2,3*2)
            
            pass
        pass
    def visit_sequenceNode(self, node):
        nodeStatus:NodeStatus = NodeStatus.VALUE_RECEIVABLE
        finalNodes = []
        nodes = []
        if node.token.type == TokenTypes.CHOICE:
            for n in node.nodes:
                    current_n = self.visit(n)
                    if current_n.type == NodeStatus.NOT_ASSIGNED_YET:
                         nodeStatus = NodeStatus.NOT_ASSIGNED_YET

                    elif current_n.type == NodeStatus.ERROR:
                         nodeStatus = NodeStatus.ERROR
                         print("Error in sequence visitor for CHOICE")
                         break 
                    
                    if current_n.node.token.type != TokenTypes.FAIL:
                         nodes.append(current_n.node)  

            if(len(nodes) == 0):
                return VisitorNode(nodeStatus,FailNode(Token(TokenTypes.FAIL,TokenTypes.FAIL.value)))
            if(len(nodes) == 1):
                 return VisitorNode(nodeStatus,nodes[0].node) 
            return VisitorNode(nodeStatus,SequenceNode(node.token, nodes))
        
       
        else:
            '''
            Visit something for tuple/array sequences.
            What it does it visits the nodes of the tuple/array and
            stores them in a nodes list.
            Choices (1|2) will be appended as such: [NN(1),NN(2)]

            Integers, Tuple/Array etc (1,2) , 3 as such:
            Tuple/Array:  [SN( NN(1), NN(2) )]        Integers:  [NN(1)]
            This Type of Converting only occures if the first element is a choice in

            a tuple/array sequence.
            ill-formed tuple/array : ( 1, (2|3) )
            ok: ((2|3), 1, (4,5))
            This should be the outcome in get_choiceSequence():  (2,1,4,5) | (3,1,4,5)
            '''

            nodes = []
            if node.nodes[0].token.type == TokenTypes.CHOICE:
                for n in node.nodes:
                    current_n = self.visit(n)
                    if current_n.node.token.type == TokenTypes.CHOICE:
                        nodes.append(current_n.node.nodes)
                    else: nodes.append([current_n.node])
                return self.get_choiceSequence(nodes)
            else: 
                for n in node.nodes:
                    current_n = self.visit(n)
                    if current_n.node.token.type == TokenTypes.FAIL:
                        # If one node fails in tuple/array, return FailNode
                        return VisitorNode(nodeStatus.VALUE_RECEIVABLE, FailNode(Token(TokenTypes.FAIL,TokenTypes.FAIL.value)))          
                    nodes.append(current_n.node)
                   
                return VisitorNode(nodeStatus.VALUE_RECEIVABLE, SequenceNode(Token(TokenTypes.ARRAY_TYPE,TokenTypes.ARRAY_TYPE.value),nodes))          


    '''
    Gets the nodes from sequence visitor.
    Turns a tuple/array sequence of tuples containing choices into a
    choice sequence.
    Example:
    
    Tuple/Array with choices: (8|39) , (9|40)
    Resulting choice Sequence: ( (8,9) | (8,40) | (39,9) | (39,40) ).
    '''
    def get_choiceSequence(self,nodes):
        finalnodes = []
        combined_nodes = list(itertools.product(*nodes))
        for cd in combined_nodes:
            current_nodes = []
            for cd2 in cd:
                if cd2.token.type == TokenTypes.ARRAY_TYPE or cd2.token.type == TokenTypes.TUPLE_TYPE:
                    for cd3 in cd2.nodes:
                        current_nodes.append(cd3)
                else: current_nodes.append(cd2) 
            finalnodes.append(current_nodes)
        return VisitorNode(NodeStatus.VALUE_RECEIVABLE,SequenceNode(Token(TokenTypes.CHOICE, TokenTypes.CHOICE.value), finalnodes))

       



    def visit_indexingNode(self, node):
        pass



    def visit_failNode(self, node):
        return VisitorNode(NodeStatus.VALUE_RECEIVABLE, node)
        




    

class BlockNode(BaseNode):
    def __init__(self, nodes:list[BaseNode]) -> None:
        self.nodes:list[BaseNode] = nodes
         
    

class ProgramNode(BaseNode):
    def __init__(self, node:BlockNode) -> None:
        self.node = node

class ScopeNode(BaseNode):
    def __init__(self, token:Token, nodes:list[BaseNode], type) -> None: #Change into Variable/IdentifierNode
        super().__init__(token)
        self.nodes = nodes
        self.type = type



class NumberNode(BaseNode):
    def __init__(self, token:Token) -> None:
        super().__init__(token)
        self.value = token.value

    def __repr__(self) -> str:
         return str(self.value)

class OperatorNode(BaseNode):
    def __init__(self, token:Token, leftNode: BaseNode, rightNode: BaseNode) -> None:
        super().__init__(token)
        self.leftNode = leftNode
        self.rightNode = rightNode

class UnaryNode(BaseNode):
     def __init__(self, token:Token, node) -> None:
        super().__init__(token)
        self.node = node

class IdentifierNode(BaseNode):
    def __init__(self, token:Token) -> None: #Change into Variable/IdentifierNode
        super().__init__(token)
        
class BindingNode(BaseNode):
    def __init__(self,token:Token, leftNode:IdentifierNode, rightNode:BaseNode) -> None:
        super().__init__(token)
        self.leftNode = leftNode
        self.rightNode = rightNode

class TypeNode(BaseNode):
    def __init__(self, token:Token) -> None: 
        super().__init__(token)
        self.type = type

class SequenceTypeNode(TypeNode):
    def __init__(self, token:Token, types:list[TypeNode]) -> None: 
        super().__init__(token)
        self.types = types


class ArgumentsNode: 
    def __init__(self, nodes:list[BaseNode]) -> None: 
        self.nodes = nodes

class FuncCallNode:
    def __init__(self,identifier:IdentifierNode, args:ArgumentsNode) -> None:
        self.identifier = identifier
        self.args = args

class ParamsNode:
    def __init__(self, nodes:list[ScopeNode]) -> None:
        self.nodes = nodes

class FuncDeclNode:
    def __init__(self,identifier:IdentifierNode, nodes:list[ParamsNode],usesLambda:bool, type:TypeNode, block:BlockNode) -> None:
        self.identifier = identifier
        self.nodes = nodes
        self.usesLambda = usesLambda
        self.type = type
        self.block = block

class ForNode(BaseNode):
     def __init__(self, token:Token, node: BaseNode, condition: BaseNode, expr: BaseNode, do: BaseNode) -> None:
        super().__init__(token)
        self.node = node
        self.condition = condition
        self.expr = expr
        self.do = do

class IfNode(BaseNode):
     def __init__(self, token:Token, if_node: BaseNode, then_node: BaseNode, else_node: BaseNode) -> None:
        super().__init__(token)
        self.if_node = if_node
        self.then_node = then_node
        self.else_node = else_node

class RigidEqNode(BaseNode):
     def __init__(self, token:Token, left_node:BaseNode, right_node:BaseNode) -> None:
        super().__init__(token)
        self.left_node = left_node
        self.right_node = right_node

class FlexibleEqNode(BaseNode):
     def __init__(self, token:Token, left_node:BaseNode, right_node:BaseNode) -> None:
        super().__init__(token)
        self.left_node = left_node
        self.right_node = right_node

class SequenceNode(BaseNode):
     def __init__(self, token:Token, nodes:list[BaseNode]) -> None:
        super().__init__(token)
        self.nodes = nodes
        self.seperator = ","
        
     def __repr__(self) -> str:
        if self.token.type == TokenTypes.CHOICE:
            self.seperator = self.token.value     
        return self.seperator.join([repr(n) for n in self.nodes])

class IndexingNode(BaseNode):
      def __init__(self, token:Token,identifier:IdentifierNode, index:BaseNode) -> None:
        super().__init__(token)
        self.identifier = identifier
        self.index = index

class FailNode(BaseNode): # Technically not need, since Fail node is 1 to 1 a BaseNode
      def __init__(self, token:Token) -> None:
        super().__init__(token)


In [258]:


class Symboltable:
    def __init__(self) -> None:
        self.symboltable = []
    
    def __info__(self) -> None:
        for t in self.symboltable:
            print("Symboltable: Name= {}, Value= {}, ParentNode= {}".format(t[0], t[1], t[2]))
    
    def check_if_exists(self, name: string, parentNode: BaseNode) -> bool:
        for symbol in self.symboltable:
            if symbol[0] == name:
                if symbol[2] == parentNode:
                    return True 
        return False
    
    def add(self, name: string, value: BaseNode, parentNode: BaseNode) -> None:
        # checks if the name already exists in the current scope. Otherwise add to table.
        if self.check_if_exists(name, parentNode) == False:
            self.symboltable.append([name, value, parentNode])
    
    def remove(self, name:string, value: BaseNode, parentNode: BaseNode) -> None:
        # checks if the table is empty.
        if len(self.symboltable) < 1:
            return
        
        # iterates through and removes the corresponding 
        for symbol in self.symboltable:
            if symbol[0] == name:
                if symbol[2] == parentNode:
                    self.symboltable.remove([name, value, parentNode]) 
    
    def get_value(self, name: string, parentNode: BaseNode) -> tuple[bool, BaseNode]:
        for symbol in self.symboltable:
            if symbol[0] == name:
                if symbol[2] == parentNode:
                    return True, symbol[1]
        return False, parentNode



In [262]:
table = Symboltable()
parentNode = NumberNode(Token(TokenTypes.INTEGER, 6))
table.remove("x", NumberNode(Token(TokenTypes.INTEGER, 7)), parentNode)
table.add("y", NumberNode(Token(TokenTypes.INTEGER, 7)), parentNode)
valid, value = table.get_value("y", parentNode)
if valid:
    print( "y: " + repr(value.visit(value).node))

y: 7


In [254]:
class Parser:
    def __init__(self, lexer: lexicon):
       self.logger: Logger = Console_Logger()
       self.end = False
       self.lexer = lexer
       self.current_token = lexer.get_token(self.lexer.current_char)
       

    def parse(self) -> BaseNode:     
        node = self.program()
        if node.hasSyntaxError or self.current_token.type != TokenTypes.EOF:
            self.logger.__log_error__("it appears there was a problem", ErrorType.SyntaxError)
        return node.node
    

       
    #####################################
    # statements
    #####################################


    """
    Checks if the program has either statements or function calls.
    Rule => statement_list | func_decl (SEMI statement_list | func_decl)*?
    """
    def program(self) -> ParsedNode:
        
        index = self.lexer.index
        token = self.current_token
        
        #node = self.func_decl()

        # checks if start of the program is not a block but a function.
        #if(node.hasSyntaxError == True):
        #    self.set_to_token(index, token)
        node = self.block()
        
        # checks if the program is not a function either.
        if(node.hasSyntaxError == True):
            return ParsedNode(None, True)
        
        nodes = node
        
        return ParsedNode(ProgramNode(nodes.node), False)
    
    

    """
    Checks if the program has a list of statements.
    Rule => statement_list
   
    def blockdelete(self) -> ParsedNode:
        
        nodes = self.statement_list()
        if(nodes[0].hasSyntaxError):
            return ParsedNode(None, True)
        return ParsedNode(BlockNode(nodes), False)
    """


    """
    Checks if there are one or more statements.
    Rule => statement (SEMI statement)*?   
    """
    def block(self) -> ParsedNode:
        node = self.statement()
        nodes = []
        nodes.append(node)

        if(node.hasSyntaxError==False):
            if(self.current_token.type == TokenTypes.SEMICOLON):
                while(self.current_token.type == TokenTypes.SEMICOLON):
                    self.forward()
                    node = self.statement()
                    if node.hasSyntaxError:
                        return ParsedNode(None,True)
                    else: nodes.append(node)
                return ParsedNode(BlockNode(nodes), False)
        return node



    """
    Checks if the current node is either an expression statemnts, if statement, for statement, a function call or an assignment statemnt.
    Rule => expr | if | for | func_call | assign_statement
    """
    def statement(self) -> ParsedNode:
        token = self.current_token
        index = self.lexer.index

        node: ParsedNode = self.nested_scope()

        # checks if current node is not an expression.
        if(node.hasSyntaxError == True):
             self.set_to_token(index,token)
             node = self.flexible_eq()
             
        if(node.hasSyntaxError == True):
            self.set_to_token(index,token)
            node = self.expr()
        
        if(node.hasSyntaxError == True):
            self.set_to_token(index,token)
            node = self.func_decl()

            
       
       
        
        
        # checks if current node is not an if statement either.
       # if(node.hasSyntaxError == True):
        #     self.set_to_token(index)
         #    node = self.for_loop()
        
        return node
    
    """
    Flexible Eq Statement (Used only to give something a value)
    Rule -> (Identifier EQUAL expr)
    """
    def flexible_eq(self)-> ParsedNode:
        left_node = self.identifier()
        if(left_node.hasSyntaxError == False):
            if(self.current_token.type == TokenTypes.EQUAL):
                token = self.current_token
                self.forward()
                right_node = self.block()
                if(right_node.hasSyntaxError == False):
                    return ParsedNode(FlexibleEqNode(token,left_node.node,right_node.node), False)
        return ParsedNode(None,True)


    """
    Rigid Eq Statement (Used only to check if two expr are equals)
    Rule -> (expr (EQUAL expr)*?)
    """
    def rigid_eq(self) -> ParsedNode:
        left_node = self.block()
        if(left_node.hasSyntaxError == False):
            if(self.current_token.type == TokenTypes.EQUAL):
                node = ParsedNode(None,True)
                while self.current_token.type == TokenTypes.EQUAL:
                    token = self.current_token
                    self.forward()
                    right_node = self.block()
                    if(right_node.hasSyntaxError == False):
                        if(node.node == None):
                                node = RigidEqNode(token,left_node.node,right_node.node)
                        if(self.current_token.type == TokenTypes.EQUAL):
                            node = RigidEqNode(token, node, right_node)
                    else: return ParsedNode(None,True)
                return ParsedNode(node,False)
            return left_node
        return ParsedNode(None,True)
   
  
    """
    Checks for a function call.
    Rule => IDENTIFIER LB (func_call_args)? RB 
    """
    def func_call(self) -> ParsedNode:
        # RULE --> IDENTIFIER LB (func_call_param)? RB  NOT IMPLEMENTED
        node = self.identifier()
        if(node.hasSyntaxError):
            return ParsedNode(None, True)  
        if(self.current_token.type == TokenTypes.LBRACKET):
            self.forward()
            
            # checks if it is an empty function call.
            if self.current_token.type == TokenTypes.RBRACKET:
                self.forward()
                return ParsedNode(FuncCallNode(node, None), False)

            params = self.func_call_args()
            for param in params:
                if param.hasSyntaxError:
                    return ParsedNode(None, True)
                
            if(self.current_token.type == TokenTypes.RBRACKET):
                self.forward()
                return ParsedNode(FuncCallNode(node, params), False)
                
        return node
        

    """
    Checks for the arguments of the function call.
    Rule => expr (COMMA expr)*?  
    """    
    def func_call_args(self) -> list[ParsedNode]:
        nodes = []
        nodes.append(self.expr())

        while True:
            if(self.current_token.type == TokenTypes.COMMA):
                self.forward()
                nodes.append(self.expr())
            else:
                for node in nodes:
                    if node.hasSyntaxError:
                        return [ParsedNode(None, True)]
                break
        
        return nodes


    """
    Checks for the declaration of a function.
    Rule => IDENTIFIER LB func_dec_param RB (COLON type)? BINDING block
           |IDENTIFIER BINDING LB nested_scope LAMBDA expr RB    
    """
    def func_decl(self) -> ParsedNode:
        identifier = self.identifier()
        if identifier.hasSyntaxError == False:

            if self.current_token.type == TokenTypes.LBRACKET:
                self.forward()

                params = self.func_decl_param()

                if self.current_token.type == TokenTypes.RBRACKET:
                    self.forward()
                    type = None
                    if(self.current_token.type == TokenTypes.COLON):
                        self.forward()
                        type = self.type()
                        if(type.hasSyntaxError):
                            return ParsedNode(None,True)
                        
                    if self.current_token.type == TokenTypes.BINDING:
                        self.forward()
                        block = self.block()
                        if(block.hasSyntaxError == False):
                            return ParsedNode(FuncDeclNode(identifier,params,False,type,block),False)
                return ParsedNode(None,True)
            
            if self.current_token.type == TokenTypes.BINDING:
                self.forward()
                if self.current_token.type == TokenTypes.LBRACKET:
                    self.forward()
                    type = None
                    params = self.func_decl_param()
                    if(params.hasSyntaxError):
                        return ParsedNode(None,True)
                    
                    if self.current_token.type == TokenTypes.LAMBDA:
                        self.forward()                   
                        block = self.block()
                        if(block.hasSyntaxError == False and self.current_token.type == TokenTypes.RBRACKET):
                            self.forward()
                            return ParsedNode(FuncDeclNode(identifier,params,True,type,block),False)
                    return ParsedNode(None,True)

        return ParsedNode(None,True)

    """
    Checks for the arguments of the function declaration.
    Rule => nested_scope
    """
    def func_decl_param(self) -> ParsedNode:
        nodes:list[ScopeNode] = []
        if(self.current_token.type == TokenTypes.RBRACKET):
            return ParsedNode(ParamsNode(nodes),False)
        
        node = self.scope()
        if(node.hasSyntaxError == False):
            nodes.append(node.node)
            while(self.current_token.type == TokenTypes.COMMA):
                self.forward()
                node = self.scope()
                if(node.hasSyntaxError):
                    return ParsedNode(None,True)
                nodes.append(node.node)
            return ParsedNode(ParamsNode(nodes),False)
        return ParsedNode(None,True)

    """
    Checks if the statement is an if statement.
    Rule => IF LB expr RB THEN CBL block CBR ELSE CBL block CBR
          | IF LB expr RB THEN expr ELSE expr  
    """
    def if_statement(self) -> ParsedNode:
        token = self.current_token

        if token.type != TokenTypes.IF:
            return ParsedNode(None, True)
        
        self.forward()
        if self.current_token.type != TokenTypes.LBRACKET:
            return ParsedNode(None, True)
        
        self.forward()
        if_node = self.rigid_eq()
        if if_node.hasSyntaxError == True or self.current_token.type != TokenTypes.RBRACKET:
            return ParsedNode(None, True)
        
        self.forward()
        if self.current_token.type != TokenTypes.THEN:
            return ParsedNode(None, True)
        
        self.forward()

        hasCB:bool = self.current_token.type == TokenTypes.CBL
        then_node = ParsedNode(None,True)
        else_node = ParsedNode(None,True)

        if(hasCB):
            self.forward()
            then_node = self.block()
            if(then_node.hasSyntaxError):
                return ParsedNode(None,True)
            if(self.current_token.type == TokenTypes.CBR):
                self.forward()
            else: return ParsedNode(None,True)
        else:
            then_node = self.expr()
            if(then_node.hasSyntaxError):
                return ParsedNode(None,True)
            
        if self.current_token.type != TokenTypes.ELSE:
            return ParsedNode(None, True)     
        self.forward()

        if(hasCB and self.current_token.type == TokenTypes.CBL):
            self.forward()
            else_node = self.block()
            if(then_node.hasSyntaxError):
                return ParsedNode(None,True)
            if(self.current_token.type == TokenTypes.CBR):
                self.forward()
            else: return ParsedNode(None,True)
        else:
            else_node = self.expr()
            if(then_node.hasSyntaxError):
                return ParsedNode(None,True)
        """
        --Issues with brackets--
        
        token = self.current_token
        index = self.lexer.index
        then_node = self.block()

        if then_node.hasSyntaxError:
            self.set_to_token(index, token)
            if self.current_token.type != TokenTypes.CBL:
                return ParsedNode(None, True)
            
            self.forward()
            then_node = self.expr()
            if then_node.hasSyntaxError and self.current_token.type != TokenTypes.CBR:
                return ParsedNode(None, True)
        
            self.forward()
            if self.current_token.type != TokenTypes.ELSE:
                return ParsedNode(None, True)
            self.forward()
            if self.current_token.type != TokenTypes.CBL:
                return ParsedNode(None, True)
            
            self.forward()
            else_node = self.expr()
            if else_node.hasSyntaxError and self.current_token.type != TokenTypes.CBR:
                return ParsedNode(None, True)
            return ParsedNode(IfNode(token, if_node.node, then_node.node, else_node.node),False)

        if self.current_token.type != TokenTypes.ELSE:
            return ParsedNode(None, True)
        
        self.forward()
        else_node = self.block()

        if else_node.hasSyntaxError:
            return ParsedNode(None, True)
        """
        return ParsedNode(IfNode(token, if_node.node, then_node.node, else_node.node),False)
            

    """
    Checks the 'Then' part of the if-statement.
    Rule => expr | (CBL block CBR)
    
    def then_statement(self) -> ParsedNode:
        pass


    
    Checks the 'else' part of the if-statement.
    Rule => expr | (CBL block CBR)
    
    def else_statement(self) -> ParsedNode:
        pass
    """



    """
    Checks if the statement is a loop expression.
    Rule => FOR CBL (scope|expr) (;expr)*? CBR
          | FOR LB (scope|expr) (,expr)*? RB DO expr
    """
    def for_loop(self) -> ParsedNode:
        token = self.current_token

        if token.type != TokenTypes.FOR:
            return ParsedNode(None, True)
        
        self.forward()
        # checks if the for loop is defined with a curly bracket.
        if self.current_token.type == TokenTypes.CBL:
            self.forward()
            return self.for_loop_curly()
        elif self.current_token.type == TokenTypes.LBRACKET:
            self.forward()
            return self.for_loop_bracket()

        return ParsedNode(None, True)
            
            
    
    def for_loop_curly(self) -> ParsedNode:
        token = self.current_token
        index = self.lexer.index

        node = self.scope()
        condition: ParsedNode
        expression: ParsedNode

        # checks if the loop content is not a scope but an expression.
        if node.hasSyntaxError:
            self.set_to_token(index, token)
            node = self.expr()
        
        # checks if the loop input is invalid.
        if node.hasSyntaxError:
            return ParsedNode(None, True)
        
        if self.current_token.type == TokenTypes.CBR:
            self.forward()
            return ParsedNode(ForNode(TokenTypes.FOR, node.node, None, None,None), False)
        
        self.forward()
        condition = self.expr()

        if condition.hasSyntaxError:
            return ParsedNode(None, True)

        if self.current_token.type == TokenTypes.SEMICOLON:
            self.forward()
            expression = self.expr()
        
            if self.current_token.type != TokenTypes.CBR or expression.hasSyntaxError:
                return ParsedNode(None, True)
        
            return ParsedNode(ForNode(TokenTypes.FOR, node.node, condition.node, expression.node),False)
        
        if self.current_token.type == TokenTypes.CBR:
            self.forward()
            return ParsedNode(ForNode(TokenTypes.FOR, node.node, None, condition.node, None), False)
        
        return ParsedNode(None, True)
    
    def for_loop_bracket(self) -> ParsedNode:
        token = self.current_token
        index = self.lexer.index
        
        node = self.scope()
        condition: ParsedNode
        expression: ParsedNode

        # checks if the loop content is not a scope but an expression.
        if node.hasSyntaxError:
            self.set_to_token(index, token)
            node = self.expr()
        
        # checks if the loop input is invalid.
        if node.hasSyntaxError:
            return ParsedNode(None, True)
        
        if self.current_token == TokenTypes.RBRACKET:
            self.forward()
            return ParsedNode(ForNode(TokenTypes.FOR, node.node, None, None,None), False)
        
        self.forward()
        condition = self.expr()

        if condition.hasSyntaxError:
            return ParsedNode(None, True)

        if self.current_token.type == TokenTypes.SEMICOLON:
            self.forward()
            expression = self.expr()
        
            if self.current_token.type != TokenTypes.RBRACKET or expression.hasSyntaxError:
                return ParsedNode(None, True)
        
            self.forward()
            if self.current_token.type != TokenTypes.DO:
                return ParsedNode(None, True)
        
            self.forward()
            do = self.expr()
        
            if do.hasSyntaxError:
                return ParsedNode(None, True)
            
            return ParsedNode(ForNode(token, node.node, condition.node, expression.node, do.node), False)
        
        if self.current_token.type != TokenTypes.RBRACKET:
                return ParsedNode(None, True)
        
        self.forward()
        if self.current_token.type != TokenTypes.DO:
            return ParsedNode(None, True)
        
        self.forward()
        do = self.expr()
        
        if do.hasSyntaxError:
            return ParsedNode(None, True)
            
        return ParsedNode(ForNode(token, node.node, None, condition.node, do.node), False)

    """
    Checks if the statement is a nested scope.
    Rule =>  (Identifier COMMA Identifier)* COLON TYPE
    """
    def nested_scope(self) -> ParsedNode:
        nodes: list[ParsedNode] = []
        nodes.append(self.identifier())

        if nodes[0].hasSyntaxError:
            return ParsedNode(None, True)
            
        hasComma:bool = False
        while True:        
            if self.current_token.type == TokenTypes.COMMA:
                hasComma = True
                self.forward()
                nodes.append(self.identifier())
            else:
                for node in nodes:
                    if node.hasSyntaxError:
                        return ParsedNode(None, True)
                break
        
        if hasComma == False or self.current_token.type != TokenTypes.COLON:
            return ParsedNode(None, True)
        
        self.forward()
        type = self.type()

        if type.hasSyntaxError == True:
            return ParsedNode(None, True)

        return ParsedNode(ScopeNode(TokenTypes.COLON, nodes, type), False)
        

    #####################################
    # expressions
    #####################################

    def expr(self) -> ParsedNode:         
        leftNode = self.choice()

        if leftNode.hasSyntaxError:
            return ParsedNode(None, True)

        if self.current_token.type != TokenTypes.DOT:
            return leftNode
        
        self.forward()
        if self.current_token.type != TokenTypes.DOT:
            return ParsedNode(None, True)
        
        token = self.current_token
        self.forward()
        rightNode = self.choice()

        if rightNode.hasSyntaxError:
            return ParsedNode(None, True)

        return ParsedNode(OperatorNode(token, leftNode.node, rightNode.node), False)
    
    
    def choice(self):
        node = self.operation()
        token = self.current_token

        nodes:list[BaseNode] = []
     
        if(node.hasSyntaxError==False and self.current_token.type == TokenTypes.CHOICE):
                nodes.append(node.node)
                while(self.current_token.type == TokenTypes.CHOICE):
                    token = self.current_token
                    self.forward()
                    node = self.operation()
                    if(node.hasSyntaxError==False):
                        nodes.append(node.node)
                    else: return ParsedNode(None,True)
                return ParsedNode(SequenceNode(token,nodes), False)
        return node

    
    """
    This method checks if a token any of the following operations: =, <, >, <=, >=, |, +, -
    Since all of this operations have the same priority and same values output, it is not needed to write them in different methods
    """
    def operation(self):
        # RULE --> op: term ((GT|LT|GE|LE|EQUAL|CHOICE|PLUS|MINUS) term)*

        left_node = self.term()

        # Checks if left node has been received and if the following token is one of the following tokens: : =, <, >, <=, >=, |, +, -

        if(left_node.hasSyntaxError == False and (self.check_type(self.current_token.type,
                [TokenTypes.GREATER,TokenTypes.GREATEREQ,TokenTypes.LOWER,TokenTypes.LOWEREQ, TokenTypes.PLUS,
                TokenTypes.MINUS]))):

                node = ParsedNode(None,True)
                
                # The while method "concatenates" the operations

                while(self.check_type(self.current_token.type,
                [TokenTypes.GREATER,TokenTypes.GREATEREQ,TokenTypes.LOWER,TokenTypes.LOWEREQ, TokenTypes.PLUS,
                TokenTypes.MINUS])):
                
                    token = self.current_token
                    self.forward()
                    right_node = self.term()
                    if(right_node.hasSyntaxError):
                        return right_node
                    
                    # Binds found operation to its left node
                    if(node.node == None):
                       node = ParsedNode(OperatorNode(token,left_node.node,right_node.node),False)
                    else: node = ParsedNode(OperatorNode(token,node.node,right_node.node),False)
                return node
        return left_node

    """
    Checks the same way in operation method but here it checks for *, /
    """
    def term(self) -> ParsedNode:
        # RULE --> factor ((MUL|DIV) factor)*
        
        left_node = self.factor() 

        if(left_node.hasSyntaxError == False and (self.check_type(self.current_token.type,[TokenTypes.MULTIPLY, TokenTypes.DIVIDE]))):
            node = ParsedNode(None,True)

             # The while method "concatenates" the operations
            while(self.check_type(self.current_token.type,[TokenTypes.MULTIPLY, TokenTypes.DIVIDE])):
               
                token = self.current_token
                self.forward()
                right_node = self.factor()
                if(right_node.hasSyntaxError):
                    return right_node
                
                # Binds found operation to its left node
                if(node.node == None):
                  node = ParsedNode(OperatorNode(token,left_node.node,right_node.node),False)
                else: node = ParsedNode(OperatorNode(token,node.node,right_node.node),False)
            return node
        return left_node
    
    """
    Checks for unary operations, Integers, brackets (highest priority)
    RULE -->  INTEGER  
        : brackets
        : (MINUS|PLUS) arith
        : func_call x() x
        : indexing     NOT IMPLEMENTING
        : --> means the same as (brackets|unary|func_call) just like in operation()
        only that for each if a different Node may be created not such as only OperationNode like in operation()
    """
    def factor(self) -> ParsedNode:
        token = self.current_token
        index = self.lexer.index

        #Integer check
        if(token.type == TokenTypes.INTEGER):
            self.forward()
            return ParsedNode(NumberNode(token),False)
        
        if(token.type == TokenTypes.FAIL):
            self.forward()
            return ParsedNode(FailNode(token),False)
        
        #Unary operation check
        if(self.check_type(self.current_token.type,[TokenTypes.PLUS, TokenTypes.MINUS])):
            self.forward()
            node = self.operation()
            if(node.hasSyntaxError):        # (--) --> Error needs (-- expr) or (--3)
                return ParsedNode(None, True)
            return ParsedNode(UnaryNode(token,node.node),False)
        
        #brackets check
        node = self.brackets()

        #if node has failed check for loop
        if(node.hasSyntaxError):
            self.set_to_token(index,token)
            node = self.indexing()

        #if node has failed check for loop
        if(node.hasSyntaxError):
            self.set_to_token(index,token)
            node = self.scope()

         #if node has failed check for loop
        if(node.hasSyntaxError):
            self.set_to_token(index,token)
            node = self.binding()

        #if node has failed check indexing
        if(node.hasSyntaxError):
            self.set_to_token(index,token)
            node = self.func_call()
        
        #if node has failed check for loop
        if(node.hasSyntaxError):
            self.set_to_token(index,token)
            node = self.for_loop()

        #if node has failed check if statement
        if(node.hasSyntaxError):
            self.set_to_token(index,token)
            node = self.if_statement()
        
        if(node.hasSyntaxError):
            self.set_to_token(index,token)
            node = self.sequence()
            
        if(node.hasSyntaxError):
            self.set_to_token(index,token)
            node = self.identifier()
        return node
    
    """
    Checks for brackets (highest priority)
    RULE --> brackets: LB expr RB
    """
    def brackets(self) -> ParsedNode: 
        if(self.current_token.type == TokenTypes.LBRACKET):
            self.forward()
            node = self.block()
        
            if(self.current_token.type == TokenTypes.RBRACKET):
                self.forward()
                return node
        return ParsedNode(None,True)
        


    """
    y := 8 y:=(x:int)  y:= method(...)...
    RULE --> scope BINDING expr
    """
    def binding(self) -> ParsedNode:
        left_node = self.identifier()

        if(left_node.hasSyntaxError == False):
            if(self.check_type(self.current_token.type,[TokenTypes.BINDING])):
                token = self.current_token
                self.forward()
                right_node = self.expr()
                if(right_node.hasSyntaxError == False):
                    return ParsedNode(BindingNode(token,left_node.node,right_node.node), False)
                else: return ParsedNode(None,True)
        return ParsedNode(None,True)

    """
    x:int
    RULE --> identifier COLON type 
    """
    def scope(self) -> ParsedNode:
        left_node = self.identifier()
        if(left_node.hasSyntaxError == False):
            if(self.check_type(self.current_token.type,[TokenTypes.COLON])):
                token = self.current_token
                self.forward()
                type = self.type()
                if(type.hasSyntaxError == False):
                    return ParsedNode(ScopeNode(token,[left_node.node],type.node),False) # Return Scope Node
                else: return ParsedNode(None,True)
        return ParsedNode(None,True)
    

    """
    variable/method name
    RULE --> identifier            NEED UPDATE
    """
    def identifier(self) -> ParsedNode:
        token = self.current_token
        if(token.type == TokenTypes.IDENTIFIER):
            self.forward()
            return ParsedNode(IdentifierNode(token), False)
        return ParsedNode(None, True) 
        
    """
    int or tuple(int,int) or array{int}
    RULE -->  INT                        
            : TUPLE LB type (,type)* RB 
    """
    def type(self) -> ParsedNode:
        # RULE -->  INT                        
        #        : TUPLE LB type (,type)* RB    

        token = self.current_token
        if(token.type == TokenTypes.INT_TYPE):
            self.forward()
            return ParsedNode(TypeNode(token),False)
        
        if(token.type == TokenTypes.TUPLE_TYPE):
            self.forward()
            if(self.current_token.type == TokenTypes.LBRACKET):
                self.forward()
                types:list[TypeNode] = []

                type = self.type()
                if(type.hasSyntaxError == False):
                    types.append(type.node)
                    if(self.check_type(self.current_token.type, [TokenTypes.COMMA])):
                        while(self.current_token.type == TokenTypes.COMMA):

                            self.forward()
                            t = self.type()

                            if(t.hasSyntaxError):  #If on error
                                return ParsedNode(None,True)
                            types.append(t.node) #else append to list of types
                     
            if(self.current_token.type == TokenTypes.RBRACKET):  
                self.forward()
                return ParsedNode(SequenceTypeNode(TokenTypes.TUPLE_TYPE,types), False)
            
        return ParsedNode(None, True) 
        
    """
    a[i:int]
    # RULE --> identifier SBL expr SBR
    """
    def indexing(self) -> ParsedNode:
        left_node = self.identifier()
        if(left_node.hasSyntaxError == False):
            if(self.current_token.type == TokenTypes.SBL):
                self.forward()
                expr_node = self.expr()
          
                if(expr_node.hasSyntaxError == False and self.current_token.type == TokenTypes.SBR):
                     self.forward()
                     return ParsedNode(IndexingNode(left_node.node.token,left_node.node,expr_node.node),False)
                return ParsedNode(None,True)
        return ParsedNode(None,True)
    
    """
    RUKE --> LB (expr COMMA expr)* RB                  --> tuple (n1,...)
             array CBL expr (COMMA expr)*? CBR         --> long-form syntax and singleton tuple/array array{n1} oder array{n1,...}
    """
    def sequence(self) -> ParsedNode:
        token = self.current_token

        nodes:list[BaseNode] = []

        # Tuple
        if(token.type == TokenTypes.LBRACKET):
            self.forward()
            node = self.expr()
            if(node.hasSyntaxError==False):
                nodes.append(node.node)
                while(self.current_token.type == TokenTypes.COMMA):
                    self.forward()
                    node = self.expr()
                    if(node.hasSyntaxError==False):
                        nodes.append(node.node)
                    else: return ParsedNode(None,True)
                if(len(nodes) > 1 and self.current_token.type == TokenTypes.RBRACKET):
                    self.forward()
                    return ParsedNode(SequenceNode(Token(TokenTypes.TUPLE_TYPE,TokenTypes.TUPLE_TYPE.value),nodes), False)


        # Array
        if(token.type == TokenTypes.ARRAY_TYPE):
                self.forward()
                if(self.current_token.type == TokenTypes.CBL):
                    self.forward()
                    node = self.expr()
                    if(node.hasSyntaxError == False):
                        nodes.append(node.node)
                        while(self.current_token.type == TokenTypes.COMMA):
                              self.forward()
                              node = self.expr()
                              if(node.hasSyntaxError==False):
                                nodes.append(node.node)
                              else: return ParsedNode(None,True)
                        if(self.current_token.type == TokenTypes.CBR):
                            self.forward()
                            return ParsedNode(SequenceNode(Token(TokenTypes.TUPLE_TYPE,TokenTypes.TUPLE_TYPE.value),nodes), False)
                        
        return ParsedNode(None, True)
                

    """
    Moves forward in the tokens list
    """
    def forward(self) -> None:
        print(self.current_token.__info__())
        self.lexer.forward()
        self.current_token = lexer.get_token(self.lexer.current_char)
        if self.current_token.type == TokenTypes.EOF:
            self.end = True
        

        
    """
    Checks if a type exists in the following types list
    """
    def check_type(self,type:TokenTypes,types:list[TokenTypes]) -> bool:
        return type in types
    

    """
    Sets current token back if a certain path lead to failure (Wrong syntax)
    May need it for later
    """
    def set_to_token(self,index, token): 
        self.current_token = token
        self.lexer.index = index


        """
        def if_statement(self) -> ParsedNode:
        token = self.current_token

        if token.type != TokenTypes.IF:
            return ParsedNode(None, True)
        
        self.forward()
        if self.current_token != TokenTypes.LBRACKET:
            return ParsedNode(None, True)
        
        self.forward()
        if_node = self.expr()
        if if_node.hasSyntaxError == True or self.current_token != TokenTypes.RBRACKET:
            return ParsedNode(None, True)
        
        self.forward()
        if self.current_token != TokenTypes.THEN:
            return ParsedNode(None, True)
        
        self.forward()
        then_node = self.then_statement()
        if then_node.hasSyntaxError == True or self.current_token != TokenTypes.ELSE:
            return ParsedNode(None, True)
        
        self.forward()
        else_node = self.else_statement()

        if else_node.hasSyntaxError:
            return ParsedNode(None, True)
        
        return ParsedNode(IfNode(token, if_node, then_node, else_node))
        """
        

In [255]:
class Interpreter:
    def __init__(self, parser: Parser):
        self.parser = parser

    def interpret(self):
        tree = self.parser.parse()
        
        return tree.visit(tree)

 (1|2) + ((3|4) + 5|36)
            
            (8|39) , (9|40)

            ( (8,9) | (8,40) | (39,9) | (39,40) )

  -->(1|2) + ( (8,9) | (8,40) | (39,9) | (39,40) )

      ((9,10) | (9,41) | (40,10) | (40,41))  ,  ((10,11) | (10,42) | (41,11) | (41,42))

      ((9,10) | (10,11) , (9,10) | (10,42)...)
            

( (1|8), (2|9), (3|10) )

     1|8   2,3 | 2,10

     1|(2,3) 1,2,10

block{

x := z;                       x !       0                       x = 3 0
z := 3;                          z = 3;  z scope  0             
d:int                           d scope 0                       
 f(b:int):=  (y: = b + x; d = y)        f!     b scope 0; y! d! y   
  f(1)                              f! b = 1  0 y = 4; d = 4 0     
x + d                           x + d x! d!  3 + 4

}

In [256]:



text = "((2|3), 1, (4,5))" 
lexer = lexicon(text)      
parser = Parser(lexer)      
interpreter = Interpreter(parser)
result = interpreter.interpret()
print(repr(result.node))



TokenTypes.LBRACKET: (
TokenTypes.LBRACKET: (
TokenTypes.INTEGER: 2
TokenTypes.CHOICE: |
TokenTypes.INTEGER: 3
TokenTypes.RBRACKET: )
TokenTypes.LBRACKET: (
TokenTypes.LBRACKET: (
TokenTypes.INTEGER: 2
TokenTypes.CHOICE: |
TokenTypes.INTEGER: 3
TokenTypes.RBRACKET: )
TokenTypes.COMMA: ,
TokenTypes.INTEGER: 1
TokenTypes.COMMA: ,
TokenTypes.LBRACKET: (
TokenTypes.INTEGER: 4
TokenTypes.LBRACKET: (
TokenTypes.INTEGER: 4
TokenTypes.COMMA: ,
TokenTypes.INTEGER: 5
TokenTypes.RBRACKET: )
TokenTypes.RBRACKET: )
[2, 1, 4, 5]|[3, 1, 4, 5]


# Testing Station

In [257]:
"""
e0 = NumberNode(Token(TokenTypes.INTEGER,3))
e1 = NumberNode(Token(TokenTypes.INTEGER,20))
e2 = NumberNode(Token(TokenTypes.INTEGER,10))

choice = SequenceNode(Token(TokenTypes.CHOICE,TokenTypes.CHOICE.value),[e1,e2])

add = OperatorNode(Token(TokenTypes.PLUS,TokenTypes.PLUS.value),e0,choice)

nodes = []
for c in add.rightNode.nodes:
    node = add.leftNode.token.value + c.value
    nodes.append(node)

for c in nodes:
  print(c)

a = [1,2,3]
b = [4,5,6]
xc = [a,b]
def f(xs:list) -> int:
  for x in xs:
    yield x

for c in xc:
   for s in f(c):
      print(s)
"""


      # (1|2) , (3|4), (5,6)
      # (1,2,(3|4))

import itertools

choiceToken = Token(TokenTypes.CHOICE,TokenTypes.CHOICE.value)
arrayToken = Token(TokenTypes.ARRAY_TYPE,TokenTypes.ARRAY_TYPE.value) 

n1 = NumberNode(Token(TokenTypes.INTEGER,1))
n2 = NumberNode(Token(TokenTypes.INTEGER,2))

c1 = SequenceNode(choiceToken, [NumberNode(Token(TokenTypes.INTEGER,1)),NumberNode(Token(TokenTypes.INTEGER,8))])

n3 = NumberNode(Token(TokenTypes.INTEGER,5))

c2 = SequenceNode(choiceToken, [NumberNode(Token(TokenTypes.INTEGER,2)),NumberNode(Token(TokenTypes.INTEGER,9))])
c3 = SequenceNode(choiceToken, [NumberNode(Token(TokenTypes.INTEGER, 3)), NumberNode(Token(TokenTypes.INTEGER, 10))])

a1 = SequenceNode(arrayToken, [NumberNode(Token(TokenTypes.INTEGER, 10)), NumberNode(Token(TokenTypes.INTEGER, 11))])
n4 = NumberNode(Token(TokenTypes.INTEGER,12))

array = SequenceNode(arrayToken, [c1,n3,c2,c3,a1,n4])

def f ():
    nodes = []
    if array.nodes[0].token.type == TokenTypes.CHOICE:
      for x in array.nodes:
          if x.token.type == TokenTypes.CHOICE:
              nodes.append(x.nodes)
          else: nodes.append([x])
    return nodes

# (8|39) , (9|40)
# ( (8,9) | (8,40) | (39,9) | (39,40) )  
# ((3|4),5,(6|7),(8|9),(10,11),12)



nodes = f()

def get(xs):
  finalnodes = []
  combined_nodes = list(itertools.product(*xs))
  for cd in combined_nodes:
    current_nodes = []
    for cd2 in cd:
       if cd2.token.type == TokenTypes.ARRAY_TYPE:
          for cd22 in cd2.nodes:
             current_nodes.append(cd22)
       else: current_nodes.append(cd2) 
    finalnodes.append(current_nodes)
  return SequenceNode(choiceToken, finalnodes)


seq = get(nodes)
for i in seq.nodes:
   print(i)




[1, 5, 2, 3, 10, 11, 12]
[1, 5, 2, 10, 10, 11, 12]
[1, 5, 9, 3, 10, 11, 12]
[1, 5, 9, 10, 10, 11, 12]
[8, 5, 2, 3, 10, 11, 12]
[8, 5, 2, 10, 10, 11, 12]
[8, 5, 9, 3, 10, 11, 12]
[8, 5, 9, 10, 10, 11, 12]


Testcases

Unary
--(2 + 3)
-2
+5
--4
--(-32)
+-4
-+67
